## Notebook for Bag-of-words and Artificial Neural Network

TODO: Figure out ravel() 1d array problem, Get TF to work

In [1]:
import pandas as pd
import numpy as np
import os
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from __future__ import division
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, batch_norm, dropout

In [2]:
# Set random seed
tf.set_random_seed(123)
np.random.seed(123)

In [3]:
labels = pd.read_csv('stocknews//labels.csv', header=None)

http://stackoverflow.com/questions/28382735/python-pandas-does-not-read-the-first-row-of-csv-file

`pd.read_csv` was cutting off the first row of labels

In [4]:
# Confirm size of labels to make sure data loaded correctly
labels.shape

(1989, 1)

In [5]:
str_tokens_sw = pd.read_csv('stocknews//tokens_str_all.csv', dtype=str, keep_default_na=False)

In [6]:
str_tokens_sw.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,georgia downs two russian warplanes as countri...,breaking musharraf to be impeached,russia today columns of troops roll into south...,russian tanks are moving towards the capital o...,afghan children raped with impunity official s...,150 russian tanks have entered south ossetia w...,breaking georgia invades south ossetia russia ...,the enemy combatent trials are nothing but sha...,georgian troops retreat from osettain capital ...,did the prep georgia for war with russia,...,georgia invades south ossetia if russia gets i...,al qaeda faces islamist backlash,condoleezza rice the us would not act to preve...,this is busy day the european union has approv...,georgia will withdraw 000 soldiers from iraq t...,why the pentagon thinks attacking iran is bad ...,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in series of...,visitors suffering from mental illnesses banne...,no help for mexico kidnapping surge
1,why wont america and nato help us if they wont...,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli tra...,georgian army flees in disarray as russians ad...,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zeala...,russia angered by israeli military sale to geo...,an american citizen living in ossetia blames a...,welcome to world war iv now in high definition,georgia move mistake of monumental proportions,...,israel and the us behind the georgian aggression,do not believe tv neither russian nor georgian...,riots are still going on in montreal canada be...,china to overtake us as largest manufacturer,war in south ossetia pics,israeli physicians group condemns state torture,russia has just beaten the united states over ...,perhaps the question about the georgia russia ...,russia is so much better at war,so this is what it come to trading sex for food
2,remember that adorable year old who sang at th...,russia ends georgia operation,if we had no sexual harassment we would have n...,al qa eda is losing support in iraq because of...,ceasefire in georgia putin outmaneuvers the west,why microsoft and intel tried to kill the xo 1...,stratfor the russo georgian war and the balanc...,trying to get sense of this whole georgia russ...,the us military was surprised by the timing an...,beats war drum as iran dumps the dollar,...,troops still in georgia did you know they were...,why russias response to georgia was right,gorbachev accuses of making serious blunder in...,russia georgia and nato cold war two,remember that adorable 62 year old who led you...,war in georgia the israeli connection,all signs point to the us encouraging georgia ...,christopher king argues that the us and nato a...,america the new mexico,bbc news asia pacific extinction by man not cl...
3,refuses israel weapons to attack iran report,when the president ordered to attack tskhinval...,israel clears troops who killed reuters cameraman,britain policy of being tough on drugs is poin...,body of 14 year old found in trunk latest rans...,china has moved 10 million quake survivors int...,bush announces operation get all up in russia ...,russian forces sink georgian ships,the commander of navy air reconnaissance squad...,92 of cnn readers russia actions in georgia ju...,...,elephants extinct by 2020,us humanitarian missions soon in georgia if ru...,georgia ddos came from us sources,russian convoy heads into georgia violating truce,israeli defence minister us against strike on ...,gorbachev we had no choice,witness russian forces head towards tbilisi in...,quarter of russians blame for conflict poll,georgian president says us military will take ...,2006 nobel laureate aleksander solzhenitsyn ac...
4,all the experts admit that we should legalise ...,war in south osetia 89 pictures made by russia...,swedish wrestler ara abrahamian throws away me...,russia exaggerated t

In [7]:
str_tokens_sw.shape

(1989, 25)

In [8]:
str_tokens_sw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 25 columns):
Top1     1989 non-null object
Top2     1989 non-null object
Top3     1989 non-null object
Top4     1989 non-null object
Top5     1989 non-null object
Top6     1989 non-null object
Top7     1989 non-null object
Top8     1989 non-null object
Top9     1989 non-null object
Top10    1989 non-null object
Top11    1989 non-null object
Top12    1989 non-null object
Top13    1989 non-null object
Top14    1989 non-null object
Top15    1989 non-null object
Top16    1989 non-null object
Top17    1989 non-null object
Top18    1989 non-null object
Top19    1989 non-null object
Top20    1989 non-null object
Top21    1989 non-null object
Top22    1989 non-null object
Top23    1989 non-null object
Top24    1989 non-null object
Top25    1989 non-null object
dtypes: object(25)
memory usage: 388.6+ KB


In [9]:
str_tokens_sw.describe()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
count,1989,1989,1989,1989,1989,1989,1989,1989,1989,1989,...,1989,1989,1989,1989,1989,1989,1989,1989,1989,1989
unique,1988,1989,1988,1989,1989,1988,1989,1989,1989,1989,...,1989,1989,1989,1989,1989,1989,1989,1989,1987,1987
top,monsanto uprooted germany bans cultivation of ...,north korea threatens war over seth rogen movie,putin who gave nato right to kill gaddafi,girl 15 sentenced to eight years over bullying...,catholic church will not be intimidated by sex...,from the delhi police why women deserve to be ...,breivik called the police 28 minutes before he...,australia to ban ultra skinny models,world bank rejects energy industry notion that...,cia attacks torture report fears legal action ...,...,controversial islamic school ordered out of to...,the reputation of the netherlands as the go to...,new study shows humans caused major shift in e...,russia says ukraine on brink of civil war,general strike to be held in greece,in the philippines masturbation is murder,opponents to muslim headscarves get powerful n...,woman in pakistan sentenced to death for takin...,,
freq,2,1,2,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,3,3


In [10]:
tf.set_random_seed(123)
np.random.seed(123)

In [11]:
def replace_num(element):
    return ' '.join([('numero' if k.isdigit() else k) for k in element.split()])

http://stackoverflow.com/questions/6905636/python-conditional-list-joins

Instead of a digit, use `'NUMERO'` as it is resistant to stemming/lemmatizing.  It's possible that headlines with numbers might contain some information.  This will prevent preprocessing from discarding the information.

In [12]:
str_tokens_sw['merged'] = str_tokens_sw.iloc[:, 0:].apply(lambda x: ' '.join(x.dropna().values.tolist()), axis=1)

In [13]:
# type(str_tokens['merged'][0])

In [14]:
str_tokens_sw['merged'] = str_tokens_sw['merged'].apply(replace_num)

In [15]:
from nltk.stem.porter import PorterStemmer

In [16]:
pstem = PorterStemmer()

In [17]:
def stm(element):
    return ' '.join([pstem.stem(k) for k in element.split()])

In [18]:
str_tokens_sw['merged'] = str_tokens_sw['merged'].apply(stm)

In [19]:
str_tokens_sw['merged']

0       georgia down two russian warplan as countri mo...
1       whi wont america and nato help us if they wont...
2       rememb that ador year old who sang at the open...
3       refus israel weapon to attack iran report when...
4       all the expert admit that we should legalis dr...
5       mom of miss gay man too bad he not numero year...
6       in an afghan prison the major of femal prison ...
7       man arrest and lock up for five hour after tak...
8       two elderli chines women have been sentenc to ...
9       british resid held in guantanamo bay win legal...
10      syria say it readi to put russian missil syste...
11      korea kim die in numero replac by lookalik say...
12      north korea halt denuclearis after us fail to ...
13      photo of numero year old iraqi suicid bomber w...
14      militari help for georgia is declar of war say...
15      russian prime minist vladimir putin ha accus t...
16      girl film isra soldier brutal sinc then her fa...
17      poland

Split labels and tokens into 4 folds
Fold 1 = [:499]
Fold 2 = [499:996]
Fold 3 = [996:1493]
Fold 4 = [1493:]

## Split data into train and test sets

In [20]:
train_text = str_tokens_sw.merged[0:1493] # train features

In [21]:
test_text = str_tokens_sw.merged[1493:] # test features

In [22]:
train_labels = labels[0:1493].values # train labels

In [23]:
test_labels = labels[1493:].values; # test labels

In [24]:
vectorizer = TfidfVectorizer( max_features=5000, ngram_range = ( 1, 4 ), sublinear_tf = True, max_df=0.66 )

In [25]:
# Only need text, not labels
train_x = vectorizer.fit_transform( train_text )

In [26]:
test_x = vectorizer.transform( test_text )

In [27]:
train_x.shape

(1493, 5000)

In [28]:
test_x.shape

(496, 5000)

In [29]:
train_labels.shape

(1493, 1)

In [30]:
test_labels.shape

(496, 1)

## Tensorflow ##

Set up graph and train through Tensorflow neural network
* 5 hidden layers fully connected
* Dropout
* He initialization

### Construct Graph

In [71]:
tf.reset_default_graph()

In [72]:
learning_rate = 0.005
keep_prob = 0.5

n_hidden_1 = 512
n_hidden_2 = 256
n_hidden_3 = 128
n_hidden_4 = 64
n_hidden_5 = 32

In [73]:
he_init = tf.contrib.layers.variance_scaling_initializer()
is_training = tf.placeholder(tf.bool, shape=(), name='Is_Training')

In [74]:
y = tf.placeholder(tf.int32, shape=(None), name='y')
X = tf.placeholder(tf.float32, shape=(None, 5000), name='X')

In [75]:
with tf.contrib.framework.arg_scope([fully_connected], weights_initializer=he_init, scope='DNN'):
    X_drop = dropout(X, keep_prob, is_training=is_training)
    hidden_1 = fully_connected(X_drop, n_hidden_1,
        activation_fn=tf.nn.elu, scope='Hidden_1')
    hidden_1_drop = dropout(hidden_1, keep_prob, is_training=is_training)
    
    hidden_2 = fully_connected(hidden_1_drop,
        n_hidden_2, activation_fn=tf.nn.elu, scope='Hidden_2')
    hidden_2_drop = dropout(hidden_2, keep_prob, is_training=is_training)
    
    hidden_3 = fully_connected(hidden_2_drop,
        n_hidden_3, activation_fn=tf.nn.elu, scope='Hidden_3')
    hidden_3_drop = dropout(hidden_3, keep_prob, is_training=is_training)
    
    hidden_4 = fully_connected(hidden_3_drop,
        n_hidden_4, activation_fn=tf.nn.elu, scope='Hidden_4')
    hidden_4_drop = dropout(hidden_4, keep_prob, is_training=is_training)
    
    hidden_5 = fully_connected(hidden_4_drop,
        n_hidden_5, activation_fn=tf.nn.elu, scope='Hidden_5')
    hidden_5_drop = dropout(hidden_5, keep_prob, is_training=is_training)
    logits = fully_connected(hidden_5_drop, 2, activation_fn=None, scope='Outputs')

In [76]:
with tf.name_scope('Loss'):
    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(x_entropy, name='Loss')

In [77]:
with tf.name_scope('Train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [78]:
with tf.name_scope('Eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [79]:
init = tf.global_variables_initializer()

### Execute Graph

In [80]:
n_epochs = 10001
batch_size = 25
batches = len(train_labels)//batch_size

In [81]:
train_X_array = train_x.toarray()
train_labels_1D = train_labels.ravel()

test_X_array = test_x.toarray()
test_labels_1D = test_labels.ravel()

In [83]:
%%time

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for k in range(batches):
            X_batch = train_X_array[k*batch_size:k*batch_size+batch_size]
            y_batch = train_labels_1D[k*batch_size:k*batch_size+batch_size]
            sess.run(training_op, feed_dict={is_training: True, X: X_batch, y: y_batch})
        if epoch % 100 == 0:
            acc_train = accuracy.eval(feed_dict={is_training: False, X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={is_training: False, X: test_X_array, y: test_labels_1D})
            print(epoch, "Train accuracy: ", acc_train, "Test accuracy: ", acc_test)

0 Train accuracy:  0.72 Test accuracy:  0.516129
100 Train accuracy:  0.96 Test accuracy:  0.518145
200 Train accuracy:  1.0 Test accuracy:  0.53629
300 Train accuracy:  1.0 Test accuracy:  0.53629
400 Train accuracy:  1.0 Test accuracy:  0.542339
500 Train accuracy:  1.0 Test accuracy:  0.514113
600 Train accuracy:  1.0 Test accuracy:  0.5
700 Train accuracy:  1.0 Test accuracy:  0.518145
800 Train accuracy:  1.0 Test accuracy:  0.512097
900 Train accuracy:  1.0 Test accuracy:  0.516129
1000 Train accuracy:  1.0 Test accuracy:  0.528226
1100 Train accuracy:  1.0 Test accuracy:  0.52621
1200 Train accuracy:  1.0 Test accuracy:  0.512097
1300 Train accuracy:  1.0 Test accuracy:  0.506048
1400 Train accuracy:  1.0 Test accuracy:  0.512097
1500 Train accuracy:  1.0 Test accuracy:  0.524194
1600 Train accuracy:  1.0 Test accuracy:  0.528226
1700 Train accuracy:  1.0 Test accuracy:  0.512097
1800 Train accuracy:  1.0 Test accuracy:  0.510081
1900 Train accuracy:  1.0 Test accuracy:  0.52419